# Computer Vision Feasibility Analysis
## *Updated stereo camera setup*
*Matt Schneider 10/6/25*

### What has changed since the last stereo analysis?

The code for the `StereoCameraClassifier` class has absorbed the dispersed singleton functions from the old notebook into a more reusable and clean implementation that allows for more constraint analysis combinations. This class is available on our [GitHub here](https://github.com/luckyowl20/PSS_CUAS/blob/master/StereoCameraClassifier.py).

This class builds on the `CameraClassifier` class which contains methods for evaluating single camera characteristics.

This specific analysis is similar to the last one I did, which is also on github although the code in its current state is pretty nasty and hard to understand.

In [1]:
# demonstration of new function
import sys
import os
# this is hacked af, eventually these will be converted to their own module
sys.path.append(os.path.abspath(os.path.join('..', 'analysis_supporting_code')))

from StereoCameraClassifier import StereoCamera, ConstraintSet

# establish list of constraints or leave as singleton float values
sigma_z_constraints = [1.0, 3.0]

sigma_d_constraints = None # dont care about error in disparity, will default to None, 
                           # meaning is a non limiting constraint
disparity_min_constraint = None

sigma_x_constraints = [0.2, 0.5, 1.0, 1.5, 2.0]
apparent_size_constraints = 20.0


# create a constraints object to set as the camera constraints later
constraint_set = ConstraintSet()
constraint_set.sigma_z = sigma_z_constraints
constraint_set.sigma_x = sigma_x_constraints
constraint_set.disparity_min = disparity_min_constraint
constraint_set.s_min_px = apparent_size_constraints

# set up the camera specs just the same as the standard camera class
image_sizes = [1920, 3840] # Nx
p_um = 2.9
f_mm = 6.0
baseline = 0.3

# initialize the stereo camera setup
stereo_camera = StereoCamera("Stereo cam 1", image_sizes, p_um, f_mm, baseline, constraints=constraint_set)

# run analysis with these parameters
range_m = 153 # 500ft in meters
object_size = 0.5

# only needed for FPS analysis, use the CameraClassifier class for this
# velocity = 15.0
# p_max = 5.0 

print(stereo_camera.analysis_title())
stereo_camera.run_analysis(range_m, object_size, print_results=False)

Analysis for: Stereo cam 1 at 6.0 mm lens


Zmax (m)  Disparity at Zmax (px)  \
Nx   sigma_z sigma_x disparity_min s_min_px                                     
1920 1.0     0.2     NaN           20.0        46.845                  13.250   
             0.5     NaN           20.0        29.627                  20.950   
             1.0     NaN           20.0        20.950                  29.627   
             1.5     NaN           20.0        17.105                  36.286   
             2.0     NaN           20.0        14.814                  41.900   
     3.0     0.2     NaN           20.0        51.724                  12.000   
             0.5     NaN           20.0        51.316                  12.095   
             1.0     NaN           20.0        36.286                  17.105   
             1.5     NaN           20.0        29.627                  20.950   
             2.0     NaN           20.0        25.658                  24.191   
3840 1.0     0.2     NaN           20.0        46.845                  13.250   
             0.5     NaN           20.0        29.627                  20.950   
             1.0     NaN           20.0        20.950                  29.627   
             1.5     NaN           20.0        17.105                  36.286   
             2.0     NaN           20.0        14.814                  41.900   
     3.0     0.2     NaN           20.0        51.724                  12.000   
             0.5     NaN           20.0        51.316                  12.095   
             1.0     NaN           20.0        36.286                  17.105   
             1.5     NaN           20.0        29.627                  20.950   
             2.0     NaN           20.0        25.658                  24.191   

                                             SigmaZ at Zmax (m)   HFOV  \
Nx   sigma_z sigma_x disparity_min s_min_px                              
1920 1.0     0.2     NaN           20.0                  1.0000  49.78   
             0.5     NaN           20.0                  1.0000  49.78   
             1.0     NaN           20.0                  1.0000  49.78   
             1.5     NaN           20.0                  1.0000  49.78   
             2.0     NaN           20.0                  1.0000  49.78   
     3.0     0.2     NaN           20.0                  1.2191  49.78   
             0.5     NaN           20.0                  3.0000  49.78   
             1.0     NaN           20.0                  3.0000  49.78   
             1.5     NaN           20.0                  3.0000  49.78   
             2.0     NaN           20.0                  3.0000  49.78   
3840 1.0     0.2     NaN           20.0                  1.0000  85.72   
             0.5     NaN           20.0                  1.0000  85.72   
             1.0     NaN           20.0                  1.0000  85.72   
             1.5     NaN           20.0                  1.0000  85.72   
             2.0     NaN           20.0                  1.0000  85.72   
     3.0     0.2     NaN           20.0                  1.2191  85.72   
             0.5     NaN           20.0                  3.0000  85.72   
             1.0     NaN           20.0                  3.0000  85.72   
             1.5     NaN           20.0                  3.0000  85.72   
             2.0     NaN           20.0                  3.0000  85.72   

                                             Apparent Size (px)  \
Nx   sigma_z sigma_x disparity_min s_min_px                       
1920 1.0     0.2     NaN           20.0                    6.76   
             0.5     NaN           20.0                    6.76   
             1.0     NaN           20.0                    6.76   
             1.5     NaN           20.0                    6.76   
             2.0     NaN           20.0                    6.76   
     3.0     0.2     NaN           20.0                    6.76   
             0.5     NaN           20.0                    6.76   
             1.0     NaN      

### Review of results
Using the new fancy camera sensor with a $2.9 \mu m$ pixel pitch that can handle our FPS requirements *(see FPS analysis notebook)*, we still run into issues in our ability to accurately estimate the depth of a drone at the distances we requrie. The next cell looks at the required distance between cameras to reach our target $Z_{max}=152m$.

In [2]:
# baseline optimization
# need range, sigma z, sigma x
stereo_camera.baseline_optimization(range_m)
    

Optimal Baseline (m)
Nx, HFOV      sigma_x sigma_z                      
(1920, 49.78) 0.2     1.0                    80.005
                      3.0                   240.014
              0.5     1.0                    32.002
                      3.0                    96.005
              1.0     1.0                    16.001
                      3.0                    48.003
              1.5     1.0                    10.667
                      3.0                    32.002
              2.0     1.0                     8.000
                      3.0                    24.001
(3840, 85.72) 0.2     1.0                    80.005
                      3.0                   240.014
              0.5     1.0                    32.002
                      3.0                    96.005
              1.0     1.0                    16.001
                      3.0                    48.003
              1.5     1.0                    10.667
                      3.0                    32.002
              2.0     1.0                     8.000
                      3.0                    24.001

### Results from baseline optimization
In order to hit our desired range of 500ft (152m), we need some real crazy distance between cameras based on the level of error we are going to tolerate. This means that we will not be able to put a stereo camera setup on a helmet with the purpose of estimating distance. The only reason to put multiple cameras on a single helmet is to cover a wider field of view with multiple cameras.